## Model object - LLM

In [1]:
from langchain.llms import OpenAI

# you can pass any parmaeters of openai Chat APi to LLM constructor
llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0.9)

prompt = "Hello there, are you chatGPT?"

# llm is a callable
completion = llm(prompt)

print(completion)



Hello, yes! I am chatGPT, an AI-powered chatbot. How can I assist you today?


## Prompt Object

In [31]:
from langchain.prompts import PromptTemplate

# crete an PromptTemplate object
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

prompt

PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='What is a good name for a company that makes {product}?', template_format='f-string', validate_template=True)

In [32]:
# just a simple string formatting.
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


## Combine LLM and prompt

In [4]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# an langchain takes in a LLM model and a prompt template
chain = LLMChain(llm=llm, prompt=prompt)

# you can format template here
# and you got an answer from OPENAI
chain.run("colorful socks")

'\n\nRainbow Socks Co.'

## Conversational Memory

One of the simpler forms of memory occurs in chatbots, where they remember previous conversations. There are a few different ways to accomplish this:     

- Buffer: This is just passing in the past N interactions in as context. N can be chosen based on a fixed number, the length of the interactions, or other!
- Summary: This involves summarizing previous conversations and passing that summary in, instead of the raw dialogue itself. Compared to Buffer, this compresses information: meaning it is more lossy, but also less likely to run into context length limits.
- Combination: A combination of the above two approaches, where you compute a summary but also pass in some previous interactions directly!

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

In [9]:
# return the whole hsitory as a string
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

# return chat history as a list of objects, which looks like
"""
{
    'history': [
    HumanMessage(content='Hi there!', additional_kwargs={}),
    AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}),
    HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}),
    AIMessage(content="That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?", additional_kwargs={}),
    HumanMessage(content='Tell me about yourself.', additional_kwargs={})
    ]
}
"""

# during chatting, the list history is automatically built.
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

## Memory Example

In [25]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}),
  AIMessage(content='whats up', additional_kwargs={})]}

In [33]:
llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0)
memory = ConversationBufferMemory(return_messages = True)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory= memory,
)

In [34]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [35]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!', additional_kwargs={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={})]
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


"That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?"

In [36]:
conversation.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!', additional_kwargs={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}), HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}), AIMessage(content="That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?", additional_kwargs={})]
Human: Tell me about yourself.
AI:

> Finished chain.


"Sure! I am an AI language model designed to assist humans with various tasks and provide helpful information. I was created by a team of developers who used advanced machine learning algorithms to train me on a vast amount of data. I am constantly learning and improving my abilities to better serve my users. Is there anything else you'd like to know about me?"

In [37]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi there!', additional_kwargs={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}),
  HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}),
  AIMessage(content="That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?", additional_kwargs={}),
  HumanMessage(content='Tell me about yourself.', additional_kwargs={}),
  AIMessage(content="Sure! I am an AI language model designed to assist humans with various tasks and provide helpful information. I was created by a team of developers who used advanced machine learning algorithms to train me on a vast amount of data. I am constantly learning and improving my abilities to better serve my users. Is there anything else you'd like to know about me?", additional_kwargs={})]}

In [40]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})
memory.save_context({"input": "haha"}, {"output":"that's it"})
memory.load_memory_variables({})

{'history': "Human: haha\nAI: that's it"}

In [41]:
memory = ConversationBufferWindowMemory( k=2)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})
memory.save_context({"input": "haha"}, {"output":"that's it"})
memory.load_memory_variables({})

{'history': "Human: not much you\nAI: not much\nHuman: haha\nAI: that's it"}

In [42]:
memory = ConversationBufferWindowMemory( k=3)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})
memory.save_context({"input": "haha"}, {"output":"that's it"})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: whats up\nHuman: not much you\nAI: not much\nHuman: haha\nAI: that's it"}